# Notes on Rust

## Make Jupyter knows Rust

Running this notebook requires Rust's jupyter kernel installed through `cargo` build chain which requires the Rust ecosystem.
- install system dependency `sudo pamac install cmake` 
- install the jupyter-kernel `cargo install evcxr_jupyter`
  ```
    Updating crates.io index
    Installing evcxr_jupyter v0.7.0
     Compiling libc v0.2.81
     ...
     Compiling evcxr_jupyter v0.7.0
     Finished release [optimized] target(s) in 3m 46s
    Installing /home/tilo/.cargo/bin/evcxr_jupyter
     Installed package `evcxr_jupyter v0.7.0` (executable `evcxr_jupyter`)
  ```

- activate jupyter-kernle ` ~/.cargo/bin/evcxr_jupyter --install` 
  ```
    Writing /home/tilo/.local/share/jupyter/kernels/rust/kernel.json
    Writing /home/tilo/.local/share/jupyter/kernels/rust/logo-32x32.png
    Writing /home/tilo/.local/share/jupyter/kernels/rust/logo-64x64.png
    Writing /home/tilo/.local/share/jupyter/kernels/rust/logo-LICENSE.md
    Installation complete
  ```


In [2]:
const MSG: &str = "Hello EvCxR";

In [3]:
MSG

"Hello EvCxR"

## Display rustc version

Followed [this advise: How can a Rust program access metadata from its Cargo package?](https://stackoverflow.com/questions/27840394/how-can-a-rust-program-access-metadata-from-its-cargo-package) and found function in module `std::env`.

In [4]:
std::env!("CARGO_MANIFEST_DIR", "none")

"/tmp/.tmplmGjAl"

In [5]:
std::env!("CARGO_PKG_NAME", "none")

"ctx"

In [6]:
let env_vars  = vec!("CARGO_MANIFEST_DIR","CARGO_PKG_AUTHORS","CARGO_PKG_DESCRIPTION","CARGO_PKG_HOMEPAGE","CARGO_PKG_NAME","CARGO_PKG_REPOSITORY","CARGO_PKG_VERSION","CARGO_PKG_VERSION_MAJOR","CARGO_PKG_VERSION_MINOR","CARGO_PKG_VERSION_PATCH","CARGO_PKG_VERSION_PRE");
env_vars[0]

"CARGO_MANIFEST_DIR"

In [7]:
for var in env_vars {println!("{} : {}", var, std::env!(var, "none"))}

Error: expected string literal

Don't know how to solve this for now. From type point of view both are string slices, the literal as well as array member.

In [8]:
std::any::Any::type_id(env_vars[0])

TypeId { t: 12392834530613204498 }

In [9]:
std::any::Any::type_id("CARGO_MANIFEST_DIR")

TypeId { t: 12392834530613204498 }

Let's resolve type's name.

In [10]:
std::any::type_name_of_val(env_vars[0])

Error: use of unstable library feature 'type_name_of_val'

Following `evcxr` command `:toolchain` requires toolchain "nightly" to be installed on the system.

```
    :toolchain nightly
    Toolchain: nightly
   
    #![feature(type_name_of_val)]
    let nightly_env_vars  =   vec!("CARGO_MANIFEST_DIR");
    std::any::type_name_of_val(nightly_env_vars[0])
    "str"
    
    #![feature(type_name_of_val)]
    std::any::type_name_of_val("CARGO_MANIFEST_DIR")
    "str"
```

I found file `.rustc_info.json` in the target directory where `evcxr_jupyter` executes the rust statements. Thought I could evaluate the which contains a path to information like this:
```
  "rustc 1.57.0 (f1edd0429 2021-11-29)
  binary: rustc
  commit-hash: f1edd0429582dd29cccacaf50fd134b05593bd9c
  commit-date: 2021-11-29
  host: x86_64-unknown-linux-gnu
  release: 1.57.0
  LLVM version: 13.0.0
  "
```

Following talks about JSON output of RUSTC but this applies to messages generated during compilation: https://doc.rust-lang.org/rustc/json.html. Following crate helps to read and parse such messages: https://docs.rs/cargo_metadata/0.14.2/cargo_metadata/index.html. But I looked for a library which parses `.rustc_info.json` for me.


In [11]:
:dep cargo_metadata = {version = "0.14.2"}

let metadata = cargo_metadata::MetadataCommand::new()
    .manifest_path(std::env!("CARGO_MANIFEST_DIR", "none").to_owned() + "/Cargo.toml")
    .features(cargo_metadata::CargoOpt::AllFeatures)
    .exec()
    .unwrap();
for entry in std::fs::read_dir(&metadata.target_directory)?
  {println!("{:?}", entry.unwrap().path())}


"/tmp/.tmplmGjAl/target/.rustc_info.json"
"/tmp/.tmplmGjAl/target/debug"
"/tmp/.tmplmGjAl/target/CACHEDIR.TAG"


()

In [12]:
:dep serde_json = {version = "1.0.59"}

// Open the file in read-only mode with buffer.
let file = std::fs::File::open(std::env!("CARGO_MANIFEST_DIR", "none").to_owned() 
                                          + "/target/.rustc_info.json")?;
let reader = std::io::BufReader::new(file);
/* For beginners Rust is indeed a challange (but only at compile time ;-)
   When ommiting following type annotation: `v : serde_json::Value`
   Then error: "invalid type: map, expected a string at line 1 column 1"
   When ommiting type annotation of `v` but access `v["rustc_fingerprint"]`
   Then error: "cannot infer type; type annotations needed", 
   which is already displayed by evcxr */
let v : serde_json::Value = serde_json::from_reader(reader)?;
v["rustc_fingerprint"]

Number(11635249661348444367)

## Plotting with Plotters

A copy for evaluation from [Data Analysis with Rust Notebooks](https://datacrayon.com/posts/programming/rust-notebooks/plotting-with-plotters/) by Data Crayon.

See also:
- [Crate plotters - A Rust drawing library focus on data plotting for both WASM and native applications](https://docs.rs/plotters/latest/plotters/index.html)
- [Developers Guide](https://plotters-rs.github.io/book/)

Plotters: "Because `evcxr` uses only SVG images and all types of series, so we don't need other types of backend. So we should put `default_features = false, features = ["evcxr", "all_series"]`" when dependency `plotters` is to be loaded into `evcxr`.

Function [`evcxr_figure()`](https://github.com/38/plotters/blob/26b24b1d30521af98e957a3a907d76b038e66c54/src/evcxr.rs#L31) is the integration provided by `plotters` with `evcxr` and is compiled in when [feature `evcxr` is configured](https://github.com/38/plotters/blob/26b24b1d30521af98e957a3a907d76b038e66c54/src/lib.rs#L748). It has following signature: 

> `pub fn plotters::evcxr::evcxr_figure<
     Draw: FnOnce(plotters::drawing::DrawingArea<
                    plotters::prelude::SVGBackend, 
                    plotters::coord::Shift>
                 ) -> Result<
                        (),
                        Box<dyn std::error::Error>>
    >(size: (u32, u32), draw: Draw) -> plotters::evcxr::SVGWrapper`. 

`evcxr_figure()` takes as arguments the size of the a drawing area and a [closure expression](https://doc.rust-lang.org/reference/expressions/closure-expr.html) creating an annonymous type implementing trait [`FnOnce`](https://doc.rust-lang.org/std/ops/trait.FnOnce.html) with it's method `call_once()` performing the call operation. `evcxr_figure()` returns an object of type `SVGWrapper` which implements following methods:
- `pub fn evcxr_display(&self)`,
- `pub fn style<S: Into<String>>(mut self, style: S) -> Self`

and which implements finally the `std::fmt::Debug` trait according to the convention described by [`evcxr_jupyter` for custom output](https://github.com/google/evcxr/blob/38981aeccd20f9bb1bad65cd6d47ed8898caa2a6/evcxr_jupyter/README.md#custom-output). 

In the function to create a figure to be displayed by `evcxr_jupyter` following steps are taken:
1. [The abstraction of a drawing area](https://docs.rs/plotters/latest/plotters/drawing/struct.DrawingArea.html) is hand over as variable `root` and with help of it's method [filled](https://docs.rs/plotters/latest/plotters/drawing/struct.DrawingArea.html#method.fill) in color [white](https://docs.rs/plotters/latest/plotters/style/colors/constant.WHITE.html).
2. A [helper object to create a chart context](https://docs.rs/plotters/latest/plotters/chart/struct.ChartBuilder.html) is created with its method [on](https://docs.rs/plotters/latest/plotters/chart/struct.ChartBuilder.html#method.on) returning the chart builder followed by chain of method calls:
  - [The caption of the chart with a text style](https://docs.rs/plotters/latest/plotters/chart/struct.ChartBuilder.html#method.caption) is set returning the chart builder instance.
  - [The margin size of the chart](https://docs.rs/plotters/latest/plotters/chart/struct.ChartBuilder.html#method.margin) is set returning the chart builder instance.
  - [The size of X label area](https://docs.rs/plotters/latest/plotters/chart/struct.ChartBuilder.html#method.x_label_area_size) is set returning the chart builder instance.
  - [The size of the Y label area](https://docs.rs/plotters/latest/plotters/chart/struct.ChartBuilder.html#method.y_label_area_size) is set returning the chart builder instance.
  - And finally [build the chart with a 2D Cartesian coordinate system.](https://docs.rs/plotters/latest/plotters/chart/struct.ChartBuilder.html#method.build_cartesian_2d) The function will return a `Result` object with either a [ChartContext<'a, DB, Cartesian2d<X::CoordDescType, Y::CoordDescType>](https://docs.rs/plotters/latest/plotters/chart/struct.ChartContext.html), where a data series can be rendered on, or a `DrawingAreaErrorKind<DB::ErrorType>`.
3. The [chart context](https://docs.rs/plotters/latest/plotters/chart/struct.ChartContext.html) method [**configure_mesh()**](https://docs.rs/plotters/latest/plotters/chart/struct.ChartContext.html#method.configure_mesh) initializes a mesh configuration object and returning a [MeshStyle](https://docs.rs/plotters/latest/plotters/chart/struct.MeshStyle.html). The mesh drawing can be finalized by calling the MeshStyle's method [draw](https://docs.rs/plotters/latest/plotters/chart/struct.MeshStyle.html#method.draw) which draws the configured mesh on the target plot returning a `Result` object ( either `()` or `DrawingAreaErrorKind<DB::ErrorType>`).
4. The [chart context's](https://docs.rs/plotters/latest/plotters/chart/struct.ChartContext.html) method [**draw_series()**](https://docs.rs/plotters/latest/plotters/chart/struct.ChartContext.html#method.draw_series) draws a data series returning a `Result` object with either a [series' annotation](https://docs.rs/plotters/latest/plotters/chart/struct.SeriesAnno.html) or a `DrawingAreaErrorKind<DB::ErrorType>` followed by a chain of method calls. A data series in Plotters is abstracted as an iterator of elements.
  - The [line series](https://docs.rs/plotters/latest/plotters/series/struct.LineSeries.html) object is created by method [new](https://docs.rs/plotters/latest/plotters/series/struct.LineSeries.html#method.new), which takes an iterator of points in guest coordinate system with a ShapeStyle and creates the element rendering the line plot. See following piece of code demonstrating the construct of the series is an iterator creating points of an coordinate system.
    > `for (x,y) in (-50..=50).map(|x| x as f32 / 50.0).map(|x| (x, x*x)) 
        {println!("{:.2},{:.2}", x, y)}`
    
- [The label](https://docs.rs/plotters/latest/plotters/chart/struct.SeriesAnno.html#method.label) sets the string used as a label for current series.
  - [The legend](https://docs.rs/plotters/latest/plotters/chart/struct.SeriesAnno.html#method.legend) sets the element creator function. *Note*: The creation function uses a shifted pixel-based coordinate system. And place the point (0,0) to the mid-right point of the shape.
5. The [chart context's](https://docs.rs/plotters/latest/plotters/chart/struct.ChartContext.html) method [**configure_series_labels()**](https://docs.rs/plotters/latest/plotters/chart/struct.ChartContext.html#method.configure_series_labels) configures the styles for drawing series labels in the chart and returns a [SeriesLabelStyle](https://docs.rs/plotters/latest/plotters/chart/struct.SeriesLabelStyle.html)  followed by chain of method calls:. 
  - [The background style](https://docs.rs/plotters/0.3.1/plotters/chart/struct.SeriesLabelStyle.html#method.background_style) is set with [ShapeStyle](https://docs.rs/plotters/0.3.1/plotters/style/struct.ShapeStyle.html) returning the series label of a target chart context,
  - [The style of the label series area](https://docs.rs/plotters/0.3.1/plotters/chart/struct.SeriesLabelStyle.html#method.border_style) is set with [ShapeStyle](https://docs.rs/plotters/0.3.1/plotters/style/struct.ShapeStyle.html) returning the series label of a target chart context,
  - and finally by calling the SeriesLabelStyle's method [draw](https://docs.rs/plotters/latest/plotters/chart/struct.SeriesLabelStyle.html#method.draw) which draws the series label area returning a `Result` object ( either `()` or `DrawingAreaErrorKind<DB::ErrorType>`).
6. The closure "closes" with returning an empty tuple, the [`()` type](https://doc.rust-lang.org/std/primitive.unit.html) wrapped in a [ok variant](https://doc.rust-lang.org/std/result/enum.Result.html#variant.Ok) of a `Result` object.

Note: `extern` and `use` declarations had been commented out for learning. Thus [namespace paths](https://doc.rust-lang.org/stable/reference/paths.html) are needed to be defined for each `struct` or even `trait`. For instance without either `use plotters::prelude::*;` or `use plotters::style::Color;` the compiler couldn't infer the type implementing [trait method `mix`](https://docs.rs/plotters/0.3.1/plotters/style/trait.Color.html#method.mix). So the  trait method needed to be called explicit with the instance of type [RGBAColor](https://docs.rs/plotters/0.3.1/plotters/style/struct.RGBAColor.html).

In [13]:
:dep plotters = { git = "https://github.com/38/plotters", default_features = false, features = ["evcxr", "line_series"] }
//extern crate plotters;
//use plotters::prelude::*;

let figure = plotters::evcxr::evcxr_figure((640, 480), |root| {
    root.fill(&plotters::style::WHITE)?; 
    // `?` required to overcome warning: "unused `Result` that must be used"
    let mut chart = plotters::chart::ChartBuilder::on(&root)
        .caption("y=x^2", 
                plotters::style::FontDesc::new(
                    "Arial".into(), 
                    50.into(), 
                    plotters::style::FontStyle::Normal))
        .margin(5)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d(-1f32..1f32, -0.1f32..1f32)?;

    chart.configure_mesh().draw()?;

    chart.draw_series(
        plotters::series::LineSeries::new(
            (-50..=50).map(|x| x as f32 / 50.0).map(|x| (x, x * x)),
            &plotters::style::colors::RED,)
        ).unwrap()
        .label("y = x^2")
        .legend(|(x,y)| 
            plotters::element::PathElement::new(
                vec![(x,y), (x + 20,y)],
                &plotters::style::colors::RED)
               );

    //use plotters::style::Color;
    chart.configure_series_labels()
        .background_style(plotters::style::Color::mix(&plotters::style::WHITE, 0.8))
        .border_style(&plotters::style::BLACK)
        .draw()?;
    Ok(())
});
figure

The type of the variable metadata was redefined, so was lost.
The type of the variable v was redefined, so was lost.


y=x^2
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.0
 
 
 
0.2
 
 
 
0.4
 
 
 
0.6
 
 
 
0.8
 
 
 
 
-0.8
 
 
 
-0.6
 
 
 
-0.4
 
 
 
-0.2
 
 
 
0.0
 
 
 
0.2
 
 
 
0.4
 
 
 
0.6
 
 
 
0.8
 
 
 
1.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="35,56 41,69 47,82 53,96 59,109 65,122 71,134 77,147 83,159 89,170 95,182 101,193 107,204 113,214 119,225 125,235 131,245 137,254 143,263 149,272 155,281 161,289 167,297 173,305 179,313 185,320 191,327 197,333 203,340 209,346 215,352 221,357 227,362 233,367 239,372 245,376 251,380 257,384 263,388 269,391 275,394 281,397 287,399 293,401 299,403 305,405 311,406 317,407 323,408 329,408 335,408 341,408 347,408 353,407 359,406 365,405 371,403 377,401 383,399 389,397 395,394 401,391 407,388 413,384 419,380 425,376 431,372 437,367 443,362 449,357 455,352 461,346 467,340 473,333 479,327 485,320 491,313 497,305 503,297 509,289 515,281 521,272 527,263 533,254 539,245 545,235 551,225 557,214 563,204 569,193 575,182 581,170 587,159 593,147 599,134 605,122 611,109 617,96 623,82 629,69 635,56 "/>
 
 
 
y = x^2

Wanted to save some time by preinstalling `plotters` ... hmm, I misunderstood what plotters is ;-):

```
cargo install plotters --no-default-features --features "evcxr,line_series"
    Updating crates.io index
error: there is nothing to install in 'plotters v0.3.1', because it has no binaries
'cargo install' is only for installing programs, and can't be used with libraries.
To use a library crate, add it as a dependency in a Cargo project instead.`
```

Here is the example of `evcxr_jupyter` for cutom output which demonstrates the convention described to draw `figure` above and to "draw" `m` below.

In [14]:
pub struct Matrix<T> {pub values: Vec<T>, pub row_size: usize}
impl<T: std::fmt::Debug> Matrix<T> {
    pub fn evcxr_display(&self) {
        let mut html = String::new();
        html.push_str("<table>");
        for r in 0..(self.values.len() / self.row_size) {
            html.push_str("<tr>");
            for c in 0..self.row_size {
                html.push_str("<td>");
                html.push_str(&format!("{:?}", self.values[r * self.row_size + c]));
                html.push_str("</td>");
            }
            html.push_str("</tr>");
        }
        html.push_str("</table>");
        println!("EVCXR_BEGIN_CONTENT text/html\n{}\nEVCXR_END_CONTENT", html);
    }
}
let m = Matrix {values: vec![1,2,3,4,5,6,7,8,9], row_size: 3};
m

1,2,3
4,5,6
7,8,9


## Plotting with Plotly

Plotly has a [jupyter notebook example](https://github.com/igiagkiozis/plotly/blob/b8144ad116aa8aae2d1a626bed18ec008d1f283e/plotly/examples/jupyter_notebook_examples.ipynb)

Method [`notebook_display()`](https://github.com/igiagkiozis/plotly/blob/b8144ad116aa8aae2d1a626bed18ec008d1f283e/plotly/src/plot.rs#L314) is the integration provided by `plotly` with `evcxr` when running a jupyter-notebook instead of jupyter-lab. It utilizes also `evcxr_jupyter`'s convention for custom output and generates HTML output.

Note: `extern` and `use` declarations had been commented out for learning. Thus [namespace paths](https://doc.rust-lang.org/stable/reference/paths.html) are needed to be defined for each `struct` or even `trait`.

Crate `plotly` provides a number of [re-exports](https://docs.rs/plotly/0.7.0/plotly/index.html#reexports) of its important stuctures, enumerations, and traits:
- pub use crate::[<span class="struct" style="color:#ad448e">layout::Layout</span>](https://docs.rs/plotly/0.7.0/plotly/layout/struct.Layout.html "struct.Layout");
- pub use crate::[<span class="enum" style="color:#508157">plot::ImageFormat</span>](https://docs.rs/plotly/0.7.0/plotly/plot/enum.ImageFormat.html "enum.ImageFormat");
- pub use crate::plot::Plot;
- pub use crate::bar::Bar;
- pub use crate::box_plot::BoxPlot;
- pub use crate::candlestick::Candlestick;
- pub use crate::contour::Contour;
- pub use crate::heat_map::HeatMap;
- pub use crate::histogram::Histogram;
- pub use crate::ohlc::Ohlc;
- pub use crate::scatter::Scatter;
- pub use crate::scatter_polar::ScatterPolar;
- pub use crate::surface::Surface;
- pub use crate::common::color::NamedColor;
- pub use crate::common::color::Rgb;
- pub use crate::common::color::Rgba;
- pub use crate::[<span class="trait" style="color:#7c5af3">plot::Trace</span>](https://docs.rs/plotly/0.7.0/plotly/plot/trait.Trace.html "trait.Trace");

In [15]:
:dep plotly = { version = ">=0.7.0" }
//extern crate plotly;

let x : Vec<i32> = vec![1,2,3,4,5,6,7,8,9,10];
let y = x.iter().map(|v| (*v as f32).powf(2.0))
    .collect::<Vec<f32>>();
let trace = plotly::Scatter::new(x,y)
    .mode(plotly::common::Mode::Markers);

let layout = plotly::Layout::new()
    .width(640)
    .height(480);

let mut plot = plotly::Plot::new();

plot.add_trace(trace);
plot.set_layout(layout);

plot.notebook_display();

The type of the variable figure was redefined, so was lost.
The type of the variable m was redefined, so was lost.


Another example working with labels, and a legend similar what we saw with plotters. The example is taken from [finishing the visualization page](https://datacrayon.com/posts/programming/rust-notebooks/finishing-touches-for-visualisation/) in which Dr. Shahin Rostami introduced his DARN crate (*D*ata *A*nalysis with *R*ust *N*otebooks), a toolbox to ease plotting with `plotly`. His [function `darn::show_plot(plot)`](https://github.com/shahinrostami/darn/blob/c1209a5dbd466803f90ca1003d88d4fa9570a27a/src/lib.rs#L13) is today replaced by `plotly`'s support for jupyter-notebooks:  `notebook_display()`.

In [16]:
:dep plotly = { version = ">=0.7.0" }

let trace1 = plotly::Scatter::new(vec![1, 2, 3, 4], vec![10, 15, 13, 17])
    .name("trace1")
    .mode(plotly::common::Mode::Markers);
let trace2 = plotly::Scatter::new(vec![2, 3, 4, 5], vec![16, 5, 11, 9])
    .name("trace2")
    .mode(plotly::common::Mode::Lines);
let trace3 = plotly::Scatter::new(vec![1, 2, 3, 4], vec![12, 9, 15, 12])
    .name("trace3");

let layout = plotly::Layout::new()
    .x_axis(
        plotly::layout::Axis::new()
            .title(plotly::common::Title::new("x axis")))
    .y_axis(
        plotly::layout::Axis::new()
            .title(plotly::common::Title::new("y axis")))
    .margin(
        plotly::layout::Margin::new()
            .top(0)
            .bottom(40)
            .left(40)
            .right(10))
    .legend(
        plotly::layout::Legend::new()
            .x(0.5)
            .y(1.1)
            .orientation(plotly::common::Orientation::Horizontal)
            .x_anchor(plotly::common::Anchor::Center));

let mut plot = plotly::Plot::new();

plot.add_trace(trace1);
plot.add_trace(trace2);
plot.add_trace(trace3);
plot.set_layout(layout);

plot.notebook_display();

##  Multidimensional Arrays and Operations with NDArray

This is based on Dr. Shahin Rostami (2020-02-29) introduction to crate [`ndarray`](https://docs.rs/ndarray/0.15.4/ndarray/) to do [Data Analysis with Rust Notebooks](https://datacrayon.com/posts/programming/rust-notebooks/multidimensional-arrays-and-operations-with-ndarray/) by Data Crayon.

> The `ndarray` crate provides an n-dimensional container for general elements and for numerics.
> 
> In n-dimensional we include, for example, 1-dimensional rows or columns, 2-dimensional matrices, and higher dimensional arrays. If the array has n dimensions, then an element in the array is accessed by using that many indices. Each dimension is also called an *axis*.
> 
> The docs for [ArrayBase](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html) provide an overview of the n-dimensional array type. Other good pages to look at are the documentation for the macros:
> - [`s![]`](https://docs.rs/ndarray/0.15.4/ndarray/macro.s.html) (Slice argument constructor) 
> - and [`azip!()`](https://docs.rs/ndarray/0.15.4/ndarray/macro.azip.html) (Array zip macro: lock step function application across several arrays and producers. This is a shorthand for `Zip`).

Here is collection of items from the docs to start me up:
> - [`ArrayBase`](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html): The n-dimensional array type itself. It is used to implement both the *owned arrays* and the *views* (an array view represents an array or a part of it, created from an [iterator](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html#loops-producers-and-iterators "Loops, Producers and Iterators"), [subview](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html#subviews "Subviews") or [slice](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html#slicing "Slicing") of an array.); see its docs for an overview of all array features.
> - The `ArrayBase<S, D>` is parameterized by `S` for the data container and `D` for the dimensionality. Type aliases [`Array`](https://docs.rs/ndarray/0.15.4/ndarray/type.Array.html) (the main specific array type owning its elements), `ArcArray`, `CowArray`, [`ArrayView`](https://docs.rs/ndarray/0.15.4/ndarray/type.ArrayView.html), and `ArrayViewMut` refer to `ArrayBase` with different types for the data container: arrays with different kinds of ownership or different kinds of array views.
> - The dimensionality of the array determines the number of **axes**, for example a 2D array has two *axes*. These are listed in “big endian” order, so that the greatest dimension is listed first, the lowest dimension with the most rapidly varying index is the last.
> - In a 2D array the index of each element is `[row, column]` as seen in this 4 × 3 example:
> ```
    [[ [0, 0], [0, 1], [0, 2] ],  // row 0
     [ [1, 0], [1, 1], [1, 2] ],  // row 1
     [ [2, 0], [2, 1], [2, 2] ],  // row 2
     [ [3, 0], [3, 1], [3, 2] ]]  // row 3
    //    \       \       \
    //   column 0  \     column 2
    //            column 1
>  ```
> - The number of *axes* for an array is fixed by its `D` type parameter: `Ix1` for a 1D array, `Ix2` for a 2D array etc. The dimension type `IxDyn` allows a dynamic number of axes.
> - A fixed size array (`[usize; N]`) of the corresponding dimensionality is used to index the `Array`, making the syntax array `[[ i, j, …]]`
>   ```
    let mut array = ndarray::Array2::zeros((4, 3));
    array[[1, 1]] = 7;
>   ```
> - The default memory order of an array is *row major* order (a.k.a “c” order), where each row is contiguous in memory. A *column major* (a.k.a. “f” or fortran) memory order array has columns (or, in general, the outermost axis) with contiguous elements.
> - The logical order of any array’s elements is the *row major* order (the rightmost index is varying the fastest). The iterators `.iter()`, `.iter_mut()` always adhere to this order, for example.
> - Arrays support all **arithmetic operations** the same way: they apply elementwise. Since the trait implementations are hard to overview, here is a summary.
>   - [Binary Operators with Two Arrays](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html#binary-operators-with-two-arrays)
>   - [Binary Operators with Array and Scalar](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html#binary-operators-with-array-and-scalar)
>   - [Unary Operators](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html#unary-operators)
>   - [Broadcasting](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html#broadcasting)
>   - [Conversions Between Array Types](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html#conversions-between-array-types)
>   - [Conversions Between Arrays and Vecs/Slices/Scalars](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html#conversions-between-arrays-and-vecsslicesscalars)
>   - [Conversions from Nested Vecs/Arrays](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html#conversions-from-nested-vecsarrays)

In above small code snippet to create a two dimesional mutable `array` has been create by calling [`ndarry::Array2`](https://docs.rs/ndarray/0.15.4/ndarray/type.Array2.html), it is a type definition and actually creates an object of type `Array` with dimension `Ix2`: `pub type Array2<A> = Array<A, Ix2>;` but `Array` has no method `zeros()`. `ArrayBase` has a method [`zeros()`](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html#method.zeros). Right, `Array` is a type definiton too: `pub type Array<A, D> = ArrayBase<OwnedRepr<A>, D>;`, "an array that owns its data uniquely."

Interesting to study structure [`OwnedRepr`](https://docs.rs/ndarray/0.15.4/ndarray/struct.OwnedRepr.html) for Rust-Newbee like me - a *pointer* of type [NonNull](https://doc.rust-lang.org/std/ptr/struct.NonNull.html):
```
pub struct OwnedRepr<A> {
    ptr: NonNull<A>,
    len: usize,
    capacity: usize,
}
```
`NonNull` itself is a structure too with a constant pointer of generic type `* const T` and a lot of attributes:
```
#[stable(feature = "nonnull", since = "1.25.0")]
#[repr(transparent)]
#[rustc_layout_scalar_valid_range_start(1)]
#[rustc_nonnull_optimization_guaranteed]
pub struct NonNull<T: ?Sized> {
    pointer: *const T,
}
```

Let's go.

### NDArray Creating

- Constructor methods for one-dimensional arrays.
- Constructor methods for two-dimensional arrays.
- Constructor methods for n-dimensional arrays.

First of all we depend on `ndarray`.

In [17]:
:dep ndarray = {version = "0.13.1"}

Create NDArrays with help of
- constructor methods
  - Create an array with [copies](https://doc.rust-lang.org/nightly/core/clone/trait.Clone.html) of `elem`, shape `shape`.
  - Create an array with zeros, shape `shape`.
  - Create an array with ones, shape `shape`.
  - Create an array with [default value](https://doc.rust-lang.org/nightly/core/default/trait.Default.html), shape `shape`.
  - Create an array with values created by function `f` 
    called with no argument 
    returning the element to create, shape `shape`.
  - Create an array with values created by the function `f`
    called with the index of the element to create, shape `shape`.
  - Create an array with the given shape from a vector.
    (No cloning of elements needed.)
  - Create an array with uninitalized elements, shape `shape`.
- a function
- a marco

In [18]:
// same as `ndarray::Array2::<f64>::ones((4,4))`
ndarray::Array::<f64, ndarray::Ix2>::from_elem((4,4), 1f64)

[[1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0]], shape=[4, 4], strides=[4, 1], layout=C (0x1), const ndim=2

In [19]:
// same as `ndarray::Array3::<f32>::zeros((3,3,3))`
ndarray::Array::<f32, ndarray::Ix3>::default((3,3,3))

[[[0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],

 [[0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],

 [[0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]]], shape=[3, 3, 3], strides=[9, 3, 1], layout=C (0x1), const ndim=3

In [20]:
ndarray::arr2(&[[1.,2.,3.],
                [4.,5.,6.]])

[[1.0, 2.0, 3.0],
 [4.0, 5.0, 6.0]], shape=[2, 3], strides=[3, 1], layout=C (0x1), const ndim=2

In [21]:
ndarray::array![[0.,0.,0.],
                [0.,0.,0.]]

[[0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0]], shape=[2, 3], strides=[3, 1], layout=C (0x1), const ndim=2

### NDArray Working

Let us work with dimensions, shapes, and index of arrays.

We depend on `ndarray` for sake of completness and create to NDArrays assigning them to variables.

In [22]:
:dep ndarray = {version = "0.13.1"}

let data_1d : ndarray::Array1::<f32> = ndarray::array![1.,2.,3.];

let data_2d : ndarray::Array2::<f32> = ndarray::array![[1.,2.,3.],
                                                       [4.,5.,6.]];

assert_eq!(data_1d.len(), 3);
assert_eq!(data_2d.len(), 6);

assert_eq!(data_2d.len_of(ndarray::Axis(1)), 3);

println!("# Dimension\n");
println!("Ix1 Array {:?}", data_1d.dim());
println!("Ix2 Array {:?}", data_2d.dim());
println!("");
println!("# Shape\n");
println!("Ix1 Array {:?}", data_1d.shape());
println!("Ix2 Array {:?}", data_2d.shape());
println!("");
println!("# Strides\n");
println!("Ix1 Array {:?}", data_1d.strides());
println!("Ix2 Array {:?}", data_2d.strides());
println!("");
println!("# Index\n");
println!("First cell of 1D {:?}", data_1d[0]);
println!("First cell of 2D {:?}", data_2d[[0, 0]]);
println!("Second cell of 1D {:?}", data_1d[1]);
println!("Second cell of 2D first row {:?}", data_2d[[0, 1]]);
println!("Last cell of 1D {:?}", data_1d[data_1d.len() - 1]);
println!("Last cell of 2D first row {:?}", data_2d[[0, data_2d.len_of(ndarray::Axis(1)) - 1]]);

# Dimension

Ix1 Array 3
Ix2 Array (2, 3)

# Shape

Ix1 Array [3]


In [23]:
:dep ndarray = {version = "0.13.1"}

let a_2d : ndarray::Array2::<f32> = ndarray::arr2(&[[1., 2.],
                                                    [3., 4.]]);

// Methods For Arrays

/* map by reference */
assert_eq!( a_2d.map(|x: &f32| -> f32 {x.powi(2)}), ndarray::arr2(&[[1.,  4.],
/* map by value     */                                              [9., 16.]]) );
assert_eq!( (&a_2d * &a_2d).mapv(f32::sqrt), a_2d );

// To understand primitive type f32's method: fn sqrt(self) -> f32
assert_eq!( 2f32.sqrt(), std::f32::consts::SQRT_2 );

Ix2 Array [2, 3]

# Strides

Ix1 Array [1]
Ix2 Array [3, 1]

# Index

First cell of 1D 1.0
First cell of 2D 1.0
Second cell of 1D 2.0
Second cell of 2D first row 2.0
Last cell of 1D 3.0
Last cell of 2D first row 3.0


### NDArray Viewing

Let us view into an array with help of [`ArrayView`](https://docs.rs/ndarray/0.15.4/ndarray/type.ArrayView.html) created from an [iterator](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html#loops-producers-and-iterators "Loops, Producers and Iterators"), [subview](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html#subviews "Subviews") or [slice](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html#slicing "Slicing") of an array.

Where as a `Array` has a [`OwnedRepr`(esentation)](https://docs.rs/ndarray/0.15.4/ndarray/struct.OwnedRepr.html) of the data a `ArrayView` has a [`ViewRepr`(esentation) ](https://docs.rs/ndarray/0.15.4/ndarray/struct.ViewRepr.html) of the data.

We depend on `ndarray` for sake of completness and create to NDArrays assigning them to variables.

#### Iterating

> Using `Zip` is the most general way to apply a procedure across one or several arrays or producers.
>
> `NdProducer` is like an iterable but for multidimensional data. All producers have dimensions and axes, like an array view, and they can be split and used with parallelization using `Zip`.
> In general, the `zip` uses a tuple of producers ([`NdProducer`](https://docs.rs/ndarray/0.13.1/ndarray/trait.NdProducer.html) trait) that all have to be of the same shape. The `NdProducer` implementation defines what its item type is (for example if it's a shared reference, mutable reference or an array view etc).
> 
> For example, `ArrayView<A, D>` is a producer, it has the same dimensions as the array view and for each iteration it produces a reference to the array element (`&A` in this case).

In [24]:
:dep ndarray = {version = "0.13.1"}

// Interate over the rows of a 2D array
let mut a_2d_rows = ndarray::Array2::<f32>::zeros((3, 2));
for mut row in a_2d_rows.genrows_mut() {
    row.fill(1.);
}
a_2d_rows

[[1.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0]], shape=[3, 2], strides=[2, 1], layout=C (0x1), const ndim=2

In [25]:
:dep ndarray = {version = "0.13.1"}

type M = ndarray::Array2<f64>;

// Create four 2d arrays of the same size
let mut a = M::zeros((4, 2));
let b = M::from_elem(a.dim(), 1.);
let c = M::from_elem(a.dim(), 2.);
let d = M::from_elem(a.dim(), 3.);

// Example 1: Perform an elementwise arithmetic operation across
// the four arrays a, b, c, d.
println!("{:?}", a);
ndarray::Zip::from(&mut a)
    .and(&b)
    .and(&c)
    .and(&d)
    .apply(|w, &x, &y, &z| {
        *w += x + y * z;
    });
println!("{:?}", a);

[[0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0]], shape=[4, 2], strides=[2, 1], layout=C (0x1), const ndim=2
[[7.0, 7.0],


#### Slicing

ndarray macro `s![]` is a slice argument constructor. [`s![]`](https://docs.rs/ndarray/0.15.4/ndarray/macro.s.html) takes a list of ranges/slices/indices/new-axes, separated by comma, with optional step sizes that are separated from the range by a semicolon. It is converted into a `SliceInfo` instance.

> The syntax is `s![ [ elem [, elem [ , … ] ] ] ]`, where `elem` is any of the following:
> - **index** : an `index` to use for taking a subview with respect to that axis. (The index is selected. The axis is removed except with .slice_collapse().)
> - **range** : a `range` with step size 1 to use for slicing that axis.
> - **range ; step** : a `range` with step size `step` to use for slicing that axis.
> - **slice** : a `Slice` instanc to use for slicing that axis.
> - **slice ; step** : a range constructed from a `Slice` instance, multiplying the step size by `step`, to use for slicing that axis.
> - **new-axis** : a `NewAxis` instance that represents the creation of a new axis. (Except for `.slice_collapse()`, which panics on `NewAxis` elements.)


In [26]:
:dep ndarray = {version = "0.13.1"}

let a = ndarray::arr2(&[[2., 3., 4., 5.],
                        [6., 7., 8., 9.]]);

a.slice(ndarray::s![..,1..3])

 [7.0, 7.0],
 [7.0, 7.0],
 [7.0, 7.0]], shape=[4, 2], strides=[2, 1], layout=C (0x1), const ndim=2


[[3.0, 4.0],
 [7.0, 8.0]], shape=[2, 2], strides=[4, 1], layout=Custom (0x0), const ndim=2


Method [ArrayBase.slice()](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html#method.slice) returns a sliced view of the array. Internally it creates a read-only view of the array and does a [ArrayBase.slice_move()](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html#method.slice_move) method call.

In below example rustc says "`a_3d` does not live long enough" when it is sliced into a view.

In [27]:
:dep ndarray = {version = "0.13.1"}

// 2 submatrices of 2 rows with 3 elements per row, means a shape of `[2, 2, 3]`.

let a_3d : ndarray::Array3::<i32> = 
           ndarray::arr3(&[[[ 1,  2,  3],     // -- 2 rows  \_
                            [ 4,  5,  6]],    // --         /
                           [[ 7,  8,  9],     //            \_ 2 submatrices
                            [10, 11, 12]]]);  //            /
           //  3 columns ..../.../.../
assert_eq!(&a_3d.shape(), &[2, 2, 3]);

/* 
   Let’s create a slice with
   - Both of the submatrices of the greatest dimension: `..`
   - Only the first row in each submatrix: `0..1`
   - Every element in each row: `..`
*/

{
    let b_arr : ndarray::ArrayView3<i32> = 
                a_3d.slice(ndarray::s![.., 0..1, ..]);
    let c_arr : ndarray::Array3<i32> = 
                ndarray::arr3(&[[[ 1,  2,  3]],
                                [[ 7,  8,  9]]]);
    assert_eq!(b_arr, c_arr);
    assert_eq!(b_arr.shape(), &[2, 1, 3]);
}
/* 
   Without limiting the scope `{}` of `b_arr`
   We've got following error I don't know to solve (`rustc --explain E0597`):
   ```
            a_3d.slice(ndarray::s![.., 0..1, ..]);
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ borrowed value does not live long enough
            a_3d.slice(ndarray::s![.., 0..1, ..]);
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ argument requires that `a_3d` is borrowed for `'static`
   `a_3d` does not live long enough
   ```
   I guess it's a nature of `evcxr` in which a variable has a longer scope as the cell.
*/

()

### NDArray Calculating

Numerical Methods for Arrays (and Axis)
- sum
- mean
- product
- var
- std


Binary Array and Array

> Let `A` be an array or view of any kind. Let `B` be an array with *owned storage* (either `Array` or `ArcArray`). Let `C` be an array with *mutable data* (either `Array`, `ArcArray` or `ArrayViewMut`). The following combinations of operands are supported for an arbitrary binary operator denoted by `@` (it can be `+`, `-`, `*`, `/` and so on).
> - `&A @ &A` which produces a new Array
> - `B @ A` which consumes B, updates it with the result, and returns it
> - `B @ &A` which consumes B, updates it with the result, and returns it
> - `C @= &A` which performs an arithmetic operation in place

Binary Array and Scalar

> For a scalar K the following combinations of operands are supported (scalar can be on either the left or right side, but ScalarOperand docs has the detailed condtions).
> - `&A @ K` or `K @ &A` which produces a new Array
> - `B @ K` or `K @ B` which consumes B, updates it with the result and returns it
> - `C @= K` which performs an arithmetic operation in place

Unary

> Let `A` be an array or view of any kind. Let `B` be an array with *owned storage* (either `Array` or `ArcArray`). The following operands are supported for an arbitrary unary operator denoted by `@` (it can be `-` or `!`).
> - `@&A` which produces a new Array
> - `@B` which consumes B, updates it with the result, and returns it


Broadcast ([element-wise product, Schur product, Hadamard product](https://en.wikipedia.org/wiki/Hadamard_product_(matrices)#In_programming_languages))

In [28]:
:dep ndarray = {version = "0.13.1"}

let a_2d : ndarray::Array2::<f32> = ndarray::arr2(&[[1., 2.],
                                                    [3., 4.]]);

// Numerical Methods for Arrays
assert_eq!( a_2d.sum(),                      10. );
assert_eq!( a_2d.sum_axis(ndarray::Axis(0)), ndarray::arr1(&[4., 6.]) );
assert_eq!( a_2d.sum_axis(ndarray::Axis(1)), ndarray::arr1(&[3., 7.]) );

// we refernce `a` to avoid being consumed after first operation
assert_eq!( &a_2d + 1.   , ndarray::arr2(&[[2., 3.],
                                           [4., 5.]]) );
assert_eq!( &a_2d + &a_2d, ndarray::arr2(&[[2., 4.],
                                           [6., 8.]]) );
assert_eq!( &a_2d * &a_2d, ndarray::arr2(&[[1.,  4.],
                                           [9., 16.]]) );
assert_eq!( &a_2d / &a_2d, ndarray::arr2(&[[1., 1.],
                                           [1., 1.]]) );
assert_eq!( &a_2d - &a_2d, ndarray::Array2::<f32>::zeros((2, 2)) );

let a_1d : ndarray::Array1::<f32> = ndarray::array![1., 2.];

assert_eq!( &a_2d + &a_1d, ndarray::arr2(&[[2., 4.],
                                           [4., 6.]]) );
assert_eq!( &a_2d * &a_1d, ndarray::arr2(&[[1., 4.],
                                           [3., 8.]]) );
assert_eq!( &a_2d / &a_1d, ndarray::arr2(&[[1., 1.], 
                                           [3., 2.]]) );
assert_eq!( &a_2d - &a_1d, ndarray::arr2(&[[0., 0.], [2., 2.]]) );

assert_eq!( a_2d.mean_axis(ndarray::Axis(0)).unwrap(), 
            ndarray::arr1(&[2.0, 3.0]) );
assert_eq!( a_2d.mean_axis(ndarray::Axis(1)).unwrap(), 
            ndarray::arr1(&[1.5, 3.5]) );

### NDArray Display

Dr. Shahin Rostami provides following [function `darn::show_array()`](https://github.com/shahinrostami/darn/blob/c1209a5dbd466803f90ca1003d88d4fa9570a27a/src/lib.rs#L120) in his crate `darn`. Similar to `plotters`'s `evcxr_figure()`, and `plotly`'s `display_notebook()` function this function makes us of `evcxr_jupyter`'s cutom output.

The type of a NDArray container is required to implement the trait `std::fmt::Debug`. In our example type `f32` has [the implementation](https://doc.rust-lang.org/std/primitive.f32.html#impl-Debug).

In [29]:
:dep ndarray = {version = "0.13.1"}

pub fn display_ndarray<T: std::fmt::Debug>(values: ndarray::Array2<T>) {
    let mut html = String::new();
    html.push_str(&format!("<table title=\"shape={:?}, strides={:?}\">", 
                           values.shape(), 
                           values.strides(),));
    for r in 0..(values.shape()[0]) {
        html.push_str("<tr>");
        for c in 0..values.shape()[1] {
            html.push_str("<td>");
            html.push_str(&format!("{:?}", values[[r, c]]));
            html.push_str("</td>");
        }
        html.push_str("</tr>");            
    }
    html.push_str("</table>");
    println!("EVCXR_BEGIN_CONTENT text/html\n{}\nEVCXR_END_CONTENT", html);
}

let data_2d : ndarray::Array2::<f32> = 
    ndarray::array![[ 1., 2., 3., 4., 5., 6., 7., 8., 9.,10.],
                    [11.,12.,13.,14.,15.,16.,17.,18.,19.,20.]];

display_ndarray(data_2d);

1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0


### NDArray Typing

Originally I wanted to follow [Mr. Rostami's](https://datacrayon.com/posts/programming/rust-notebooks/typed-arrays-from-string-arrays-for-dataset-operation/) next page on converting parts of the Iris data set into types specific of columns. I don't understand the method selected to create a new data set [`ndarray::stack![]`](https://docs.rs/ndarray/0.15.4/ndarray/macro.stack.html): stack arrays along the new axis. The original string data is accessed with help of method [`column(ndarray::Ix)`](https://docs.rs/ndarray/0.15.4/ndarray/struct.ArrayBase.html#method.column): return an array view of column `index`.

I will go and give Slice a try for selection. 

... and ran into an issue of primitive type's `f32` [implementation](https://doc.rust-lang.org/std/primitive.f32.html#impl-FromStr) of trait `FromStr` which not only has a required method `from_str()` but also an [associated type](https://doc.rust-lang.org/std/str/trait.FromStr.html#associated-types) `Err` ...

In [30]:
:dep ndarray = {version = "0.13.1"}

let a = ndarray::arr2(&[["2.0", "3.0", "4.0", "5.0"],
                        ["6.0", "7.0", "8.0", "9.0"]]);

a.slice(ndarray::s![..,1..3]).mapv(|elem: &str| -> f32 {f32::from_str(elem).unwrap()})

Error: no function or associated item named `from_str` found for type `f32` in the current scope

As I'm still in learning mode I'd like to avoid the `use` decaration where ever possible. This makes the code not readable - as Rust's sugar is missing - but more understandable in terms of utilized crates and their items. As side effect you often cannot not use the [dot-operator](https://doc.rust-lang.org/stable/nomicon/dot-operator.html) and thus to learn "essential Rust".

In [31]:
std::str::FromStr::f32::from_str("2.0")

Error: the value of the associated type `Err` (from trait `FromStr`) must be specified

Error: ambiguous associated type

`rustc`'s first help drew my attention to associated types.

Associated types are decribed in Rust's reference with an [container example](https://doc.rust-lang.org/reference/items/associated-items.html#associated-types-container-example). The example unfortunately does not apply the sample trait implementation.

In [32]:
trait Container {
    type E;
    fn empty() -> Self;
    fn insert(&mut self, elem: Self::E);
}


impl<T> Container for Vec<T> {
    type E = T;
    fn empty() -> Vec<T> { Vec::<T>::new() }
    fn insert(&mut self, x: T) {self.push(x); }
}

 Here is one of my tries similar to the one above with `std::str::FromStr`. 

In [33]:
let mut a_vec : Vec<f32> = Container::Vec::f32::empty();

Error: the value of the associated type `E` (from trait `Container`) must be specified

Error: ambiguous associated type

In [34]:
/* 
  Following statement took me hours to find 
  after I had been misleaded by rustc's second help:
  >  help: use fully-qualified syntax
  >
  >  <dyn Container as Trait>::Vec
*/
let mut a_vec : Vec<f32> = <Vec::<f32> as Container>::empty();
<Vec::<f32> as Container>::insert(&mut a_vec, 3f32);
a_vec

[3.0]

Coming back to our original issue when we wanted to create primitive type `f32` from `str` slice without using a `use` declaration. Somehow with this statement the associated type is specified too. I guess this is because `rustc` finds the right implementation which specifies `type Err = ParseFloatError;`.

In [35]:
<f32 as std::str::FromStr>::from_str("2.0")

Ok(2.0)

In [36]:
:dep ndarray = {version = "0.13.1"}

let a : ndarray::Array2<&str> = 
        ndarray::arr2(&[["2.0", "3.0", "4.0", "5.0"],
                        ["6.0", "7.0", "8.0", "9.0"]]);

a.slice(ndarray::s![..,1..3]).mapv(
    |elem: &str| -> f32 {
        <f32 as std::str::FromStr>::from_str(elem).unwrap()
    }
)

[[3.0, 4.0],
 [7.0, 8.0]], shape=[2, 2], strides=[2, 1], layout=C (0x1), const ndim=2

## Foreign Function Interface

[FFI mdoule](https://doc.rust-lang.org/std/ffi/index.html) "provides utilities to handle data across non-Rust interfaces, like other programming languages and the underlying operating system."

Rust represents 
- owned strings with the [String](https://doc.rust-lang.org/std/string/struct.String.html) type, 
- and borrowed slices of strings with the [str](https://doc.rust-lang.org/std/primitive.str.html) primitive type. 

[The Rustonomicon](https://doc.rust-lang.org/nomicon/ffi.html) as well as [Rust by Example](https://doc.rust-lang.org/rust-by-example/std_misc/ffi.html) has sample code.

Following Rust code runs only once. As soon as EVCXR has compiled the code after jupyter-kernel start it executes. When the jupyter-cell is then revisit it generates an error complaining that C-function `fn csqrtf(z: Complex) -> Complex;` and `fn ccosf(z: Complex) -> Complex;` is redefined.

In [37]:
// this extern block links to the libm library
#[link(name = "m")]
extern {
    // this is a foreign function
    // that computes the square root of a single precision complex number
    fn csqrtf(z: Complex) -> Complex;

    fn ccosf(z: Complex) -> Complex;
}

// Since calling foreign functions is considered unsafe,
// it's common to write safe wrappers around them.
fn cos(z: Complex) -> Complex {
    unsafe { ccosf(z) }
}

// Minimal implementation of single precision complex numbers
#[repr(C)]
#[derive(Clone, Copy)]
struct Complex {
    re: f32,
    im: f32,
}

impl std::fmt::Debug for Complex {
    fn fmt(&self, f: &mut std::fmt::Formatter) -> std::fmt::Result {
        if self.im < 0. {
            write!(f, "{}-{}i", self.re, -self.im)
        } else {
            write!(f, "{}+{}i", self.re, self.im)
        }
    }
}

{
    // z = -1 + 0i
    let z = Complex { re: -1., im: 0. };

    // calling a foreign function is an unsafe operation
    let z_sqrt = unsafe { csqrtf(z) };

    println!("the square root of {:?} is {:?}", z, z_sqrt);

    // calling safe API wrapped around unsafe operation
    println!("cos({:?}) = {:?}", z, cos(z));
}


the square root of -1+0i is 0+1i
cos(-1+0i) = 0.5403023+0i


()